In [1]:
import pandas as pd
import numpy as np
import os
import time
import pickle
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [27]:
import warnings
warnings.filterwarnings('ignore')

## Login

In [7]:
synergy_url = 'https://www.synergysportstech.com'
app_url = 'https://apps.synergysports.com/basketball'

In [3]:
def login(driver, cred, synergy_url=synergy_url):
    driver.get(f'{synergy_url}/synergy/')
    time.sleep(5)
    # Get username & password
    with open(cred, 'r') as f:
        user_password = f.readlines()[0].replace('\n', '').split(', ')
    # Login
    userfield = driver.find_element_by_id('Username')
    userfield.send_keys(user_password[0])
    passwordfield = driver.find_element_by_id('Password')
    passwordfield.send_keys(user_password[1])
    driver.find_element_by_xpath("//button[@class='btn btn-primary']").click()

In [4]:
# driver = webdriver.Chrome('/usr/local/bin/chromedriver')
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)



====== WebDriver manager ======
Current google-chrome version is 110.0.5481
Get LATEST chromedriver version for 110.0.5481 google-chrome
Driver [/Users/xulian/.wdm/drivers/chromedriver/mac64/110.0.5481.77/chromedriver] found in cache


In [5]:
cred = 'synergy_cred.txt'
login(driver, cred)

/var/folders/_2/xf7b66914jxdlhgxdz6fhjyw0000gn/T/ipykernel_35278/33737124.py:8: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  userfield = driver.find_element_by_id('Username')
/var/folders/_2/xf7b66914jxdlhgxdz6fhjyw0000gn/T/ipykernel_35278/33737124.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  passwordfield = driver.find_element_by_id('Password')
/var/folders/_2/xf7b66914jxdlhgxdz6fhjyw0000gn/T/ipykernel_35278/33737124.py:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath("//button[@class='btn btn-primary']").click()


## Scrape Setup

In [46]:
downloads_dir = '../../../Downloads'

### Situations

In [11]:
situations_dict = {
    'Pick and Rolls Including Passes': '101',
    'Isolations Including Passes': '100',
    'Post-Ups Including Passes': '102',
    'Catch and Shoot': '116',
    'Isolation': '10',
    'P&R Ball Handler': '11',
    'Post-Up': '15',
    'P%R Roll Man': '31',
    'Spot Up': '50',
    'Off Screen': '67',
    'Hand Off': '78',
    'Cut': '81',
    'Offensive Rebounds': '89',
    'Transition': '92',
    'Miscellaneous': '124',
    'All field goal attempts': '99',
    'At Rim': '127',
    'Long (3 point jump shots)': '109',
    'All Jump Shots off the Dribble': '110'
}

### Functions

In [56]:
def create_dirs(league_dict):
    league = league_dict['league']
    for season in league_dict['seasonId'].keys():
        dirname = f'{league}_{season}'
        if dirname not in os.listdir():
            os.mkdir(dirname)

In [60]:
def get_stats_sheet(situation, 
                    reportType, competition, season,
                    league_dict, 
                    output_format = 'xlsx',
                    situations_dict=situations_dict,
                    downloads_dir=downloads_dir,
                    driver=driver):
    sheet_url = (
        f'{app_url}/'
        'leaderboards?'
        f'leagueId={league_dict["leagueId"]}&'
        f'seasonId={league_dict["seasonId"][season]}&'
        f'competitionIds={league_dict["competitionIds"][competition]}&'
        f'reportType={league_dict["reportType"][reportType]}&'
        f'situation={situations_dict[situation]}'
    )
    driver.get(sheet_url)
    time.sleep(5)
    
    # Download
    xpath = "//div[@class='ng-select']"
    if output_format == 'csv':
        xpath = "//div[@class='ng-select ml-2']"
        
    for b in driver.find_elements_by_xpath(xpath):
        if b.text == output_format.upper():
            b.click()
    
    time.sleep(3)
    
    # Move to the right folder
    league = league_dict["league"]
    sheet_filename = (
        'Leaderboards - '
        f'{league} '
        f'{season} '
        f'{competition} - '
        f'{situation} - '
        f'{reportType}'
        f'.{output_format}'
    )
    if sheet_filename in os.listdir(downloads_dir):
        os.rename(f'{downloads_dir}/{sheet_filename}',
                  f'{league}_{season}/{sheet_filename}')

## Scrape NBA

In [61]:
nba_dict = {
    'league': 'NBA',
    'leagueId': '54457dce300969b132fcfb34',
    'seasonId': {
        '2021-2022': '6120261cea4488c9fd5c57c5',
        '2020-2021': '5fce72f5f68e52f827c39b4c',
        '2019-2020': '5d51e0c6f52909811e13ee2d',
        '2018-2019': '5b6e102011ef0d11039af1e3',
        '2017-2018': '59af08c917120e9c9a9797ff'
    },
    'competitionIds': {
        'Regular Season': '560100ac8dc7a24394b955ee'
    },
    'reportType': {
        'Player Offensive': '0',
        'Player Defensive': '1',
        'Team Offensive': '2',
        'Team Defensive': '3'
    }
}

In [62]:
create_dirs(nba_dict)

In [63]:
## Example
get_stats_sheet(situation='Pick and Rolls Including Passes',
                reportType='Player Offensive',
                competition='Regular Season',
                season='2021-2022',
                league_dict=nba_dict,
                output_format='csv')

In [7]:
driver.get(f'{app_url}/leaderboards')

In [20]:
# driver.close()